#### Yolo image detection

In [9]:
import cv2
import numpy as np
import glob
import random


# Load Yolo
net = cv2.dnn.readNet("yolov3_training_1000.weights", "yolov3_testing.cfg")

# Name custom object
classes = ["mattn"]

# Images path
images_path = glob.glob(r"C:\Users\Younes\Desktop\yolo_custom_detection\1.jpg")



layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Insert here the path of your images
random.shuffle(images_path)
# loop through all the images
for img_path in images_path:
    # Loading image
    img = cv2.imread(img_path)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                # Object detected
                print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            cropped=img[y:y+h, x:x+w]
            label = str(classes[class_ids[i]])
            
            cv2.rectangle(img, (x, y), (x + w, y + h), (0,255,0), 2)
            cv2.rectangle(img, (x-2,y-2), (x+w, y-30), (0,255,0), cv2.FILLED)
            cv2.putText(img, label,((x+5,y-5)),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)


    cv2.imshow("Image", img)
    cv2.imshow("Mat", face)
    key = cv2.waitKey(0)

cv2.destroyAllWindows()

0
[[0]]


#### Real time YOLO Detection

In [19]:
import cv2
import numpy as np
import time
# Load Yolo
net = cv2.dnn.readNet("yolov3_training_1000.weights", "yolov3_testing.cfg")
classes = ['mattn']

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Loading camera
cap = cv2.VideoCapture("1.mp4")
font = cv2.FONT_HERSHEY_PLAIN
starting_time = time.time()
frame_id = 0
while True:
    _, frame = cap.read()
    frame=cv2.resize(frame,(480,480))
    frame_id += 1
    height, width, channels = frame.shape
        # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.rectangle(frame, (x, y), (x + w, y + 30), color, -1)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 1.5, (255,255,255), 2)

    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

#### Real time Detection & Recognition TN-license plate 

In [37]:
from PIL import Image 
from keras.applications.mobilenet import preprocess_input
import base64 
from io import BytesIO 
import json 
import random 
import cv2 
from keras.models import load_model 
import numpy as np 
threshold = 0.5 
font = cv2.FONT_HERSHEY_SIMPLEX
from tensorflow import keras
from keras.preprocessing import image 
model = keras.models.load_model('mattn.h5')

# Load Yolo
net = cv2.dnn.readNet("yolov3_training_1000.weights", "yolov3_testing.cfg")
classes = ['mattn']

layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Loading camera
cap = cv2.VideoCapture("2.mp4")
font = cv2.FONT_HERSHEY_PLAIN

while True:
    _, frame = cap.read()
    frame=cv2.resize(frame,(640,480))
    height, width, channels = frame.shape
        # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
   
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]     
            mat=framemat=frame[y:y+h, x:x+w]
            
            if type(mat) is np.ndarray: 
                mat = cv2.resize(frame, (224, 224))     
                im = Image.fromarray(mat, 'RGB') 
                img_array = np.array(im) 
                img_array_expanded_dims = np.expand_dims(img_array, axis=0) 
                img_array = preprocess_input(img_array_expanded_dims)
                pred = model.predict(img_array)
                print(pred)
                prediction = np.argmax(pred[0]) 
                #label=class_labels[pred.argmax()]
                probabilityValue =np.amax(pred)
                
                if probabilityValue > threshold:

                    if prediction==0:
                        name='notsteal :'
                    elif prediction==1:
                        name='steal :'
                        cv2.imwrite("Steal/" +'StealCar'+str(y) + ".jpg", framemat)
                        
                        
                       
                           
            text = "{}: {:.2f}%".format(name, probabilityValue * 100)
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            #cv2.rectangle(frame, (x, y), (x + w, y + 30), color, -1)
            #cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 1.5, (255,255,255), 2)
            cv2.putText(frame, text, ((x+5,y-5)),cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2) 
             
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

[[0.59984815 0.40015182]]
[[0.6088413  0.39115876]]
[[0.6084149  0.39158505]]
[[0.6072816  0.39271832]]
[[0.6082662  0.39173383]]
[[0.60416615 0.39583385]]
[[0.6093661  0.39063385]]
[[0.6124687  0.38753128]]
[[0.60740036 0.39259967]]
[[0.6110181  0.38898188]]
[[0.61521506 0.38478494]]
[[0.6192083  0.38079166]]
[[0.6166052  0.38339472]]
[[0.62091786 0.37908214]]
[[0.6218247  0.37817526]]
[[0.6214719  0.37852815]]
[[0.62085724 0.37914276]]
[[0.61558056 0.38441944]]
[[0.61506134 0.38493857]]
[[0.6146745 0.3853255]]
[[0.609992   0.39000797]]
[[0.6137398  0.38626018]]
[[0.61495924 0.38504076]]
[[0.62004566 0.37995437]]
[[0.61671627 0.3832837 ]]
[[0.61659056 0.38340944]]
[[0.6173246 0.3826754]]
[[0.61862254 0.38137752]]
[[0.6171576  0.38284242]]
[[0.615888 0.384112]]
[[0.03363244 0.9663676 ]]
[[0.03489007 0.96510994]]
[[0.03414736 0.9658527 ]]
[[0.03365101 0.966349  ]]
[[0.03406595 0.96593404]]
[[0.03397186 0.96602815]]
[[0.0339398 0.9660602]]
[[0.03390313 0.96609694]]
[[0.03403068 0.9659693